In [1]:
import os
import sys
sys.path.append('../../')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats
from matplotlib.lines import Line2D
from statsmodels.stats.multitest import multipletests
import itertools
from sklearn.utils import resample

In [5]:
from facct22.utils import get_db_con

from facct22.analysis_functions import (
    get_all_decisions, groups, schemas, users, dps, dt, pdr, group_order, colors, 
    ttests_operational_metrics, assign_conf_mat_cell, _modify_value_and_time, ttests_operational_metrics
)

In [4]:
cred_file = '../../conf/credentials.yaml'
engine = get_db_con(cred_file)

In [12]:
# comparisons = [
#     ('Control-A', 'Control-B'),
#     ('Control-B', 'LIME'),
#     ('Control-B', 'TreeInt'),
#     ('Control-B', 'TreeSHAP'),
#     ('Control-B', 'Random'),
#     ('Control-B', 'Irrelevant'),
#     ('Control-B', 'LIME'),
#     ('Control-B', 'TreeInt'),
#     ('Control-B', 'TreeSHAP'),
#     ('TreeInt', 'Random'),
#     ('TreeSHAP', 'Random'),
#     ('LIME', 'Random'),
#     ('TreeInt', 'Irrelevant'),
#     ('TreeSHAP', 'Irrelevant'),
#     ('LIME', 'Irrelevant'),
# ]

comparisons = [
    ('Control-A', 'Control-B'),
    ('Control-B', 'Explainer'),
    ('Control-B', 'Random'),
    ('Control-B', 'Irrelevant'),
    ('Random', 'Explainer'),
    ('Irrelevant', 'Explainer'),
]

param_grid = {
    'fn': [-0.8, -1, -1.2, -3, -5],
    'p_loss_trx': [0.2, 0.3, 0.4], 
    'cust_worth': [0, 1, 3, 5], 
    'p_loss_cust': [0, 0.05, 0.1, 0.3], 
    'p_return_cust': [0, 0.2, 0.3],
    'suspicious_handling': [0, 600, 1800, 3600, 'approve', 'decline'] # If this is an int, it's correct and timepenalty
}


In [6]:
def do_parameter_sweep(
    decisions, 
    comparisons,
    fn,
    p_loss_trx, 
    cust_worth,
    p_loss_cust,
    p_return_cust,
    suspicious_handling
):
    all_param_combinations = itertools.product(
        fn, 
        p_loss_trx, 
        cust_worth, 
        p_loss_cust, 
        p_return_cust,
        suspicious_handling
    )

    results = list()
    significance_results = list()
    for config in all_param_combinations:
        
        # suspicious handling
        if isinstance(config[5], int):
            add_time= config[5]
            suspicious_strategy = 'correct'
        else:
            add_time= 0 # The value doesn't matter
            suspicious_strategy = config[5]
            
        
        param = {
            'fn': config[0],
            'p_loss_trx': config[1],
            'cust_worth': config[2],
            'p_loss_cust': config[3],
            'p_return_cust': config[4],
            'suspicious_add_time': add_time,
            'suspicious_strategy': suspicious_strategy
        }

        metrics = list()
        ttests = list()
        
        df = dt(decisions, param, suspicious_strategy, ['group'])
        df['metric'] = 'dt'
        
        ttest = ttests_operational_metrics(df, comparisons)
        ttest['metric'] = 'dt'
        
        metrics.append(df)
        ttests.append(ttest)
        
        
        df = dps(decisions, param, suspicious_strategy, ['group'])
        df['metric'] = 'dps'
        
        ttest = ttests_operational_metrics(df, comparisons)
        ttest['metric'] = 'dps'
        
        metrics.append(df)
        ttests.append(ttest)
        
        df = pdr(decisions, param, suspicious_strategy, ['group'], n_samples=500, n_iterations=100)
        df['metric'] = 'pdr'
        
        ttest = ttests_operational_metrics(df, comparisons)
        ttest['metric'] = 'pdr'
        
        metrics.append(df)
        ttests.append(ttest)
        
        res = pd.concat(metrics)
        sig_res = pd.concat(ttests)
        
        # appending the parameter values
        for p, v in param.items():
            res[p] = v
            sig_res[p] = v


        results.append(res)
        significance_results.append(sig_res)

    
    return pd.concat(results), pd.concat(significance_results)

#### What if a simple model threshold was used, without a review band?  

In [10]:
def transform_score(score: float, rev_threshold: int = 39) -> int:
    """Computes the score according to the specified thresholds.

    The default values correspond to the thresholds established
    for the model.


    Parameters
    ----------
    score : float
        The non-normalized as retrieved by the model, i.e., in the
        range [0, 1], or in the range [0, 1000].

    rev_threshold : int
        The non-normalized review threshold below which any transaction
        will be automatically accepted (identified as legitimate).


    Returns
    -------
    int
        The normalized score between [0, 1000].
    """
    if 0 < score <= 1:
        score *= 1000

    # Compute the score for the given thresholds
    if score < rev_threshold:
        score = 500 * score / rev_threshold
    else:
        threshold_nnorm = 500 / (1000 - rev_threshold)
        score = threshold_nnorm * score + 1000 * (1 - threshold_nnorm)

    # Since CM multiplies by 1000, we have to divide by 1000
    return round(score)


def fetch_trx_scores(db_conn, groups, schemas):
    qs = [
        f"""
            with unique_trx as (
                select
                    trx_id, trx_score, trx_label 
                from openxai{schema[9:]}.explanation_components
                group by 1, 2, 3
            )
            select distinct on (trx_id)
                trx_id, xplz_id, (label_fields -> 'amount_usd')::int as trx_amnt, 
                trx_score, label, rf."group"
            from {schema}.trx_user_assignments join unique_trx using(trx_id)
            join {schema}.review_feedback rf using(xplz_id)
            --where rf."group" in {tuple(groups)}
            order by trx_id
        """
        for schema in schemas
    ]
    
    return pd.concat([pd.read_sql(q, db_conn) for q in qs])




def assign_conf_mat(score, label, threshold):
        if score > threshold:
            return 'tp' if label==1 else 'fp'
        else:
            return 'fn' if label==1 else 'tn'


In [11]:
fetch_trx_scores(engine, group_order, schemas)

,trx_id,xplz_id,trx_amnt,trx_score,label,group
